# io power - Samna only - v0.39.9

- Use PDM_CLK and PDM_DATA driven by FPGA to XA3:
- PDM_CLK and PDM_DATA toggle in the normal way
- only PDM_CLK toggles
- only PDM_DATA toggles
- PDM_CLK and PDM_DATA stays at all level combinations:
    - PDM_CLK=0, PDM_DATA=0
    - PDM_CLK=0, PDM_DATA=1
    - PDM_CLK=1, PDM_DATA=0
    - PDM_CLK=1, PDM_DATA=1

### Initialization and connection to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')
import numpy as np

# Open the device and connect it to the power monitor.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")

power_monitor = xylo_node.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = xylo_node.get_stop_watch()
io = xylo_node.get_io_module()

xylo_model = xylo_node.get_model()
source = samna.graph.source_to(xylo_model.get_sink_node())

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo_model.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')

# XyloCOnfiguration
xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()

## With the network, clock frequency can not go low as 6.25
clock_frequencies = [12.5, 25, 50]
ts = 1e-3

power_measurements_625 = []
power_measurements_125 = []
power_measurements_25 = []
power_measurements_50 = []

Samna version 0.39.9.0


In [2]:
disable_main_clock = False

### Encode a PDM input chirp

In [3]:
# - Encode a PDM input chirp
from rockpool.devices.xylo.syns65302 import AFESimPDM
# from rockpool.devices.xylo.syns65302.afe import PDM_SAMPLING_RATE

import numpy as np 
from scipy.signal import chirp

PDM_SAMPLING_RATE = 1 * 1000000

pdm_sr = PDM_SAMPLING_RATE
print(pdm_sr)
net_dt = 1024e-6
audio_sr = PDM_SAMPLING_RATE # 48e3

afesim_pdm = AFESimPDM.from_specification(
    select_filters=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
    spike_gen_mode="divisive_norm",
    # spike_gen_mode="threshold",
    # fixed_threshold_vec = [2**25 for i in range(16)],
    rate_scale_factor=63,
    low_pass_averaging_window=84e-3,
    dn_EPS=1,
    fixed_threshold_vec=None,
    dt=net_dt,
)

dur = 200e-3

T = int(audio_sr * dur)
f_start = 1000
f_end = 20e3
times = np.arange(0, T) / audio_sr
signal = chirp(times, f_start, T/audio_sr, f_end, method='linear', phi = 90)
# signal = np.sin(2 * np.pi * f_start * times)

__scale = 0.95
out_pdm, state_pdm, rec_pdm = afesim_pdm((signal * __scale, audio_sr))

input_pdm = (rec_pdm['0_PDMADC']['0_MicrophonePDM_output'] + 1) / 2

print(input_pdm)

PDM_SAMPLES_PER_DT = int(PDM_SAMPLING_RATE * net_dt)
num_dt = np.size(input_pdm) // PDM_SAMPLES_PER_DT

# - Bin samples into `dt`-length windows and trim
input_raster = np.reshape(
    input_pdm[: int(num_dt * PDM_SAMPLES_PER_DT)], [-1, PDM_SAMPLES_PER_DT]
)

WARNING    /home/vleite/SynSense/rockpool/rockpool/devices/xylo/syns65302/afe/pdm/microphone_pdm.py:146: UserWarning: 

+++++++++++++++++++++++++++++++++++++++++++++++++++++++ warnings +++++++++++++++++++++++++++++++++++++++++++++++++++++++
In practice, the input to the PDM microphone (fed by a clock of rate:1562500.0) is the analog audio signal.
In simulations, however, we have to use sampled audio signal at the input to mimic this analog signal.
Here we resample the input audio to the higher sampling rate of PDM microphone (1562500.0).
For a more realistic simulation, it is better to provide an audio signal which is originally sampled with a higher rate.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


  warnings.warn(
 [py.warnings]

+++++++++++++++++++++++++++++++++++++++++++++++++++++++ warnings +++++++++++++++++++++++++++++++++++++++++++++++++++++++
In practice, the input to the PDM microphone (fed by a clo

1000000
[0. 1. 1. ... 1. 0. 1.]


In [4]:
try:
    from tqdm.autonotebook import tqdm
except:
    tqdm = lambda x: x

### Configure PDM_CLK in 1 MHz and MAIN_CLK

In [5]:
def configure_clocks():

    if (disable_main_clock):
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)

        # Disable main clock
        io.write_config(0x0008, 0)

    else:
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.main_clock_frequency = 6250000
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)



### Use PDM_CLK and PDM_DATA driven by FPGA:
- No network mapped and main clock disabled
  
- PDM_CLK and PDM_DATA toggle in the normal way:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA toggles with random data, not constant zero or one, but no specific pattern requirement either

In [6]:
import time

# Feed PDM_CLK externally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Send PDM data
# - Send PDM data and extract activity
# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(i)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	520 uW


### Use PDM_CLK and PDM_DATA driven by FPGA:
- No network mapped and main clock disabled
  
- PDM_CLK only toggle:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA constant zero

In [6]:
import time

# Feed PDM_CLK externally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Send PDM data
power_events = []
for input_sample in tqdm(input_raster):
    # - Send all zeros with same size samples as the chirp
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(0)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	520 uW


### Use PDM_CLK and PDM_DATA driven by XyloAudio 3:
- No network mapped and main clock disabled
  
- PDM_CLK and PDM_DATA toggle in the normal way:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA toggles with random data, not constant zero or one, but no specific pattern requirement either

In [6]:
import time

# Feed PDM_CLK internally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Send PDM data
# - Send PDM data and extract activity
# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(i)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	520 uW


### Use PDM_CLK and PDM_DATA driven by XyloAudio 3:
- No network mapped and main clock disabled
  
- only PDM_CLK toggle:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA constant zero

In [6]:
import time

# Feed PDM_CLK internally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Send PDM data
# - Send PDM data and extract activity
# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(0)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	519 uW
